In [206]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from config import BaseConfig
config = BaseConfig()

In [207]:
data_df = pd.read_csv(config.data_csv_path)

In [208]:
from utils.loading import get_json

level = get_json(config.level_path)

In [209]:
level[21]['children'][1]

{'code': '500102', 'name': '涪陵区', 'province': '50', 'city': '01', 'area': '02'}

In [210]:
data_df[data_df['current_city'] == "重庆市"]

,current_province,current_city,current_county,current_members_live_with,gender,year_born,edu_level,hometown_code,hometown,year_current_flow,...,hometown_Name_County,hometown_lon,hometown_lat,first_Name_Province,first_Name_Prefecture,first_Name_County,first_lon,first_lat,current_lon,current_lat
48,重庆市,重庆市,开州区,4,2,1992,3,130521,河北省邢台市邢台县,2012,...,邢台县,114.146819,37.146761,NaN,NaN,NaN,NaN,NaN,108.380181,31.275899
86,重庆市,重庆市,奉节县,2,2,1954,1,500231,重庆市县垫江县,2012,...,垫江县,107.431823,30.254430,重庆市,重庆市,奉节县,109.345528,30.953272,109.345528,30.953272
91,重庆市,重庆市,沙坪坝区,2,1,1963,3,500116,重庆市市辖区江津区,2012,...,江津区,106.259395,29.030923,重庆市,重庆市,沙坪坝区,106.363202,29.627029,106.363202,29.627029
102,重庆市,重庆市,南岸区,4,2,1995,5,500240,重庆市县石柱土家族自治县,2015,...,石柱土家族自治县,108.293808,30.095058,重庆市,重庆市,南岸区,106.658250,29.541191,106.658250,29.541191
150,重庆市,重庆市,璧山区,1,2,1954,3,500109,重庆市市辖区北碚区,2015,...,北碚区,106.509337,29.862111,重庆市,重庆市,璧山区,106.187698,29.559973,106.187698,29.559973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169453,重庆市,重庆市,巫山县,3,2,1989,3,420527,湖北省宜昌市秭归县,2015,...,秭归县,110.676086,30.902233,NaN,NaN,NaN,NaN,NaN,109.897211,31.117359
169540,重庆市,重庆市,江北区,4,2,1957,2,510502,四川省泸州市江阳区,2013,...,江阳区,105.403039,28.856441,NaN,NaN,NaN,NaN,NaN,106.703732,29.615767
169793,重庆市,重庆市,九龙坡区,3,1,1992,6,500118,重庆市市辖区永川区,2016,...,永川区,105.869188,29.291497,重庆市,重庆市,大渡口区,106.453951,29.419839,106.361645,29.433123
169804,重庆市,重庆市,江北区,6,2,1957,3,511325,四川省南充市西充县,2012,...,西充县,105.846521,31.060286,黑龙江省,鸡西市,虎林市,133.102197,45.985118,106.703732,29.615767


In [211]:
def fill_current(df: pd.DataFrame, level: list) -> pd.DataFrame:
    # Initialize the columns
    df['current_province_code'] = None
    df['current_city_code'] = None
    df['current_county_code'] = None
    
    # Iterate over the rows in the DataFrame
    for idx, row in df.iterrows():
        row_province = row['current_province']
        row_city = row['current_city']
        row_county = row['current_county']
        
        # Iterate over the `level` list (assuming it's a list of dicts)
        for province in level:
            if row_province != province['name']:
                continue

            df.at[idx, 'current_province_code'] = province['province']
            # case 1: city is province, 重庆 data is malformed and is treated as a normal province
            if row_city == row_province:
                df.at[idx, 'current_city_code'] = province['children'][0]['city']
                for county in province['children']:
                    if county['name'] == row_county:
                        df.at[idx, 'current_county_code'] = county['area']
                        break
            else:
                for city in province['children']:
                    if row_city != city['name']:
                        continue

                    df.at[idx, 'current_city_code'] = city['city']
                    for county in city['children']:
                        if row_county != county['name']:
                            continue

                        df.at[idx, 'current_county_code'] = county['area']
                        break

    return df


In [212]:
# Military bases will be None
preprocessed_df = fill_current(data_df.copy(), level=level)

In [213]:
unfound_chongqing_rows = (preprocessed_df['current_city'] == '重庆市') & (preprocessed_df['current_county_code'].isna())

# chongqing has missing counties from the dataset
preprocessed_df[unfound_chongqing_rows]['current_county'].unique()


array(['秀山县', '酉阳县', '两江新区', '石柱县', '彭水县'], dtype=object)

In [214]:
unfound_cities_rows = (preprocessed_df['current_city_code'].isna())

preprocessed_df_unfound_cities = preprocessed_df.copy()[unfound_cities_rows]
preprocessed_df_unfound_cities['combined'] = preprocessed_df_unfound_cities['current_province'] + preprocessed_df_unfound_cities['current_city'] + preprocessed_df_unfound_cities['current_county']
preprocessed_df_unfound_cities['combined'].nunique()

127

In [215]:
found_citites_but_unfound_neighborhood_rows = (~preprocessed_df['current_city_code'].isna()) & (preprocessed_df['current_county_code'].isna())


missing_neighborhood_subset_preprocessed_df = preprocessed_df.copy()[found_citites_but_unfound_neighborhood_rows]
missing_neighborhood_subset_preprocessed_df['combined'] = missing_neighborhood_subset_preprocessed_df['current_province'] + missing_neighborhood_subset_preprocessed_df['current_city'] + missing_neighborhood_subset_preprocessed_df['current_county']
missing_neighborhood_subset_preprocessed_df['combined'].nunique()

153